In [3]:
date = "20250519"

In [4]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import os
import openpyxl
from collections import Counter

In [3]:
_cellines = ['A549','GM12878','H1.hESC','HeLa.S3','HepG2','HT1080','HUVEC','IMR.90','K562','MCF.7','NCI.H460','NHEK','SK.MEL.5','SK.N.DZ','SK.N.SH']

## by threshold

# <font color="blue">by cell lines</font>

In [12]:
_threshold = ["0"]

K = 6
Q = 0
kmer = [(K,Q)]

path_input = "../results/Train_one_test_all/mer%s_%s/"%(K,date)

encoding = "rawcounts"

### 1DCNN
# ae = "AE_MLP"
# _model_type = ["1DCNN"]
# encoding_dim = 512

_model_type = ["RF"]
segment = [(200,5000)]

file_type = "cell_line"
#Create output files
wb = openpyxl.Workbook()

out_file = path_input + "train_one_test_all_%smer%smiss_with_std.xlsx"%(K,Q)
wb.save(out_file)
writer = ExcelWriter(out_file, engine='openpyxl',mode='a')

all_oa = pd.DataFrame()
all_auc = pd.DataFrame()

for Cell_train in _celline:   
    
    df_stat_oa = pd.DataFrame()
    df_stat_auc = pd.DataFrame()
      
    for (m,n) in segment:
        
        for model_type in _model_type:
            
            
            for Cell_test in _celline:       

                for (K,Q) in kmer:

                    for threshold in _threshold:
                        in_file = path_input + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                        xl = pd.ExcelFile(in_file)
                        print(in_file)
                        _cell = []
                        for i in range(1,11):
                            _cell.append(xl.parse(f"eva{i}",index_col=0))
                        dff = pd.concat(_cell).groupby(level=0).mean()
                        std = pd.concat(_cell).groupby(level=0).std()
#                         cell_avg = round(dff,2).astype(str) + " ± " + round(std,2).astype(str) ## with std
                        cell_avg = round(dff,2) 
                        auc_oa = cell_avg[["OA","AUC"]]
                        auc_oa = pd.DataFrame(auc_oa.loc[0]).T
                        auc_oa.columns = ["%s_oa"%(Cell_test), "%s_auc"%(Cell_test)]
                        auc_oa.index = [f'{Cell_train}']
                        df_stat_oa = pd.concat([df_stat_oa,auc_oa["%s_oa"%(Cell_test)]],axis=1)
                        df_stat_auc = pd.concat([df_stat_auc,auc_oa["%s_auc"%(Cell_test)]],axis=1)
    all_oa = pd.concat([all_oa,df_stat_oa],axis=0)    
    all_auc = pd.concat([all_auc,df_stat_auc],axis=0)
    
all_oa.columns = [col.replace('_oa', '') for col in all_oa.columns]
all_auc.columns = [col.replace('_auc', '') for col in all_auc.columns]
all_oa.to_excel(writer,sheet_name="oa")
all_auc.to_excel(writer,sheet_name="auc")
writer.close()


../results/Train_one_test_all/mer6_20250519/10fold_train_A549_test_A549_training_length_200_5000_rawcounts_0_RF_6mer0miss_test0.xlsx
../results/Train_one_test_all/mer6_20250519/10fold_train_A549_test_GM12878_training_length_200_5000_rawcounts_0_RF_6mer0miss_test0.xlsx
../results/Train_one_test_all/mer6_20250519/10fold_train_A549_test_HeLa.S3_training_length_200_5000_rawcounts_0_RF_6mer0miss_test0.xlsx
../results/Train_one_test_all/mer6_20250519/10fold_train_GM12878_test_A549_training_length_200_5000_rawcounts_0_RF_6mer0miss_test0.xlsx
../results/Train_one_test_all/mer6_20250519/10fold_train_GM12878_test_GM12878_training_length_200_5000_rawcounts_0_RF_6mer0miss_test0.xlsx
../results/Train_one_test_all/mer6_20250519/10fold_train_GM12878_test_HeLa.S3_training_length_200_5000_rawcounts_0_RF_6mer0miss_test0.xlsx
../results/Train_one_test_all/mer6_20250519/10fold_train_HeLa.S3_test_A549_training_length_200_5000_rawcounts_0_RF_6mer0miss_test0.xlsx
../results/Train_one_test_all/mer6_20250519/1